<a href="https://colab.research.google.com/github/SojeongShin/NLP_2023/blob/Notion/7_1_%E1%84%8E%E1%85%A1%E1%84%8B%E1%85%AF%E1%86%AB_%E1%84%8E%E1%85%AE%E1%86%A8%E1%84%89%E1%85%A9_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [자연어처리]
# 7주차(7-1). 차원 축소(2)

# 202001581 신소정 <- 본인의 학번, 이름으로 변경하여 제출
* **모든 셀 실행** 후 제출하시기 바랍니다.
* **실습 (7-1-1)**이 있습니다. (제출 기한: 10/17(화) 23시 59분까지)

# 이제 Colab 사용에 모든 수강생분들이 어느정도 익숙해 졌을 거라 생각됩니다!
> ##### 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해야 합니다.





---

## 6주차 수업 내용 리뷰 -시작-

## 6.2 PCA를 이용한 차원 축소



데이터 셋 준비 (우리에게 익숙한 20newsgroups)

In [ ]:
from sklearn.datasets import fetch_20newsgroups

#20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

#학습 데이터셋을 가져옴
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
#검증 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

X_train = newsgroups_train.data   #학습 데이터셋
y_train = newsgroups_train.target #학습 데이터셋

X_test = newsgroups_test.data     #검증 데이터셋
y_test = newsgroups_test.target   #검증 데이터셋

전처리
* 토큰화, 불용어처리, 스테밍


In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

RegTok = RegexpTokenizer("[\w']{3,}") # 정규포현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) #영어 불용어를 가져옴
stemmer = PorterStemmer() # 스테머로 포터스테머 사용

def my_tokenizer(text):
    tokens = RegTok.tokenize(text)
    # stopwords 제외
    words = [word for word in tokens if word not in english_stops]
    # portr stemmer 적용
    features = [stemmer.stem(token) for token in words]
    return features

In [ ]:
# 약간의 시간(약 30초)이 소요됩니다.
from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf = TfidfVectorizer(tokenizer=my_tokenizer, max_features=2000, min_df=5, max_df=0.5)
# 토큰화 외에 아무것도 적용하지 않은 채로 확인
tfidf = TfidfVectorizer(tokenizer=my_tokenizer) ### ... 이 부분을 완성하시오
X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
print('Original tfidf matrix shape:', X_train_tfidf.shape)

Original tfidf matrix shape: (2034, 20085)


In [ ]:
# 약간의 시간(약 30초)이 소요됩니다.
from sklearn.decomposition import PCA

#  20,085 차원을 2,000 차원으로 축소
pca = PCA(n_components=2000, random_state=7) ### ... 이 부분을 완성하시오
X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

print('Original tfidf matrix shape:', X_train_tfidf.shape)
print('PCA Converted matrix shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

Original tfidf matrix shape: (2034, 20085)
PCA Converted matrix shape: (2034, 2000)
Sum of explained variance ratio: 1.000


In [ ]:
from sklearn.linear_model import LogisticRegression

LR_clf = LogisticRegression() #분류기 선언

# PCA로 차원 축소된 X 데이터 이용
LR_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))

# PCA로 차원 축소된 X 데이터 이용
LR_clf.fit(X_train_pca, y_train) ### ... 이 부분을 완성하시오
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

#Train set score: 0.962
#Test set score: 0.761
#Train set score: 0.962
#Test set score: 0.761


## 6주차 수업 내용 리뷰 -끝-

---

## 7-1 주차 수업 내용 -시작-



## 6.3 LSA를 이용한 차원 축소와 의미 파악


* 기존의 DTM이나 DTM에 단어의 중요도에 따른 가중치를 주었던 TF-IDF 행렬은 단어의 의미를 전혀 고려하지 못한다는 단점을 갖고 있었습니다.
* LSA는 기본적으로 DTM이나 TF-IDF 행렬에 절단된 SVD(truncated SVD)를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 갖고 있습니다.

### DTM

In [ ]:
corpus = [
    '먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요'
]

# 각 문서를 공백 기준으로 단어로 분리
tokenized_corpus = [doc.split() for doc in corpus]
print(tokenized_corpus)

[['먹고', '싶은', '사과'], ['먹고', '싶은', '바나나'], ['길고', '노란', '바나나', '바나나'], ['저는', '과일이', '좋아요']]


In [ ]:
# 모든 단어들의 집합 생성 후 가나다 순으로 정렬
vocabulary = sorted(set(word for doc in tokenized_corpus for word in doc))
print(vocabulary)

# 단어와 인덱스를 맵핑한 딕셔너리 생성
word2idx = {word: idx for idx, word in enumerate(vocabulary)}
print(word2idx)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']
{'과일이': 0, '길고': 1, '노란': 2, '먹고': 3, '바나나': 4, '사과': 5, '싶은': 6, '저는': 7, '좋아요': 8}


In [ ]:
# 빈도수 계산
word_frequency = []
for doc in tokenized_corpus:
    frequency = [0] * len(vocabulary)
    for word in doc:
        if word in word2idx:
            frequency[word2idx[word]] += 1
    word_frequency.append(frequency)

for freq in word_frequency:
    print(freq)

[0, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 0, 2, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 1]


### sklearn의 CountVectorizer를 이용한 DTM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    '먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요'
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
A = vector.fit_transform(corpus).toarray()

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)
print(A)
print(A.shape)

{'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}
[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
(4, 9)


### Full SVD

* 4 × 9의 크기를 가지는 DTM이 생성되었습니다.-> 4개의 문서, 9개의 단어

* 이에 대해서 풀 SVD(full SVD)를 수행해보겠습니다.
    * 단, 여기서는 대각 행렬의 변수명을 Σ가 아니라 S를 사용합니다.
    * 또한 V의 전치 행렬을 VT라고 하겠습니다.
    * 소수점의 길이가 너무 길게 출력하면 보기 힘들어서 두번째 자리까지만 출력하기위해서 .round(2)를 사용합니다.

In [ ]:
import numpy as np

U, s, VT = np.linalg.svd(A, full_matrices = True)

In [ ]:
print('행렬 U :')
print(U.round(2))
print('행렬 U의 크기(shape) :',np.shape(U))

행렬 U :
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기(shape) : (4, 4)


* 4 × 4의 크기를 가지는 직교 행렬 U가 생성되었습니다.
* 이제 대각 행렬 S를 확인해봅시다.

In [ ]:
print('특이값 벡터 :')
print(s.round(2))
print('특이값 벡터의 크기(shape) :',np.shape(s))

특이값 벡터 :
[2.69 2.05 1.73 0.77]
특이값 벡터의 크기(shape) : (4,)


* Numpy의 linalg.svd()는 특이값 분해의 결과로 대각 행렬이 아니라 특이값의 리스트를 반환합니다.
* 그러므로 앞서 본 수식의 형식으로 보려면 이를 다시 대각 행렬로 바꾸어 주어야 합니다.
* 우선 특이값을 s에 저장하고 대각 행렬 크기의 행렬을 생성한 후에 그 행렬에 특이값을 삽입해도록 하겠습니다.

In [ ]:
# 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성 <- 원래 벡터로 복원하는 과정
S = np.zeros((4, 9))

# 특이값을 대각행렬에 삽입
S[:4, :4] = np.diag(s)

print('대각 행렬 S :')
print(S.round(2))

print('대각 행렬의 크기(shape) :')
print(np.shape(S))

대각 행렬 S :
[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
대각 행렬의 크기(shape) :
(4, 9)


* 4 × 9의 크기를 가지는 대각 행렬 S가 생성되었습니다.
* 2.69 > 2.05 > 1.73 > 0.77 순으로 값이 내림차순을 보이는 것을 확인할 수 있습니다.
* 정보량이 클수록 첫번째 벡터, 숫자가 크다.

In [ ]:
print('직교행렬 VT :')
print(VT.round(2))

print('직교 행렬 VT의 크기(shape) :')
print(np.shape(VT))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
직교 행렬 VT의 크기(shape) :
(9, 9)


* 9 × 9의 크기를 가지는 직교 행렬 VT(V의 전치 행렬)가 생성되었습니다.
<br><br>
* 이제 U × S × VT를 하면 기존의 행렬 A가 나와야 합니다.
* Numpy의 allclose()는 2개의 행렬이 동일하면 True를 리턴합니다.
    * 이를 사용하여 정말로 기존의 행렬 A와 동일한지 확인해보겠습니다.

In [ ]:
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

### Truncated SVD

* 지금까지 수행한 것은 풀 SVD(Full SVD)입니다.
* 이제 t를 정하고, 절단된 SVD(Truncated SVD)를 수행해보도록 합시다. 여기서는 t=2로 하겠습니다.
* 우선 대각 행렬 S 내의 특이값 중에서 상위 2개만 남기고 제거해보도록 하겠습니다.

In [ ]:
# 특이값 상위 2개만 보존
S = S[:2,:2] ### ... 이 부분을 완성하시오

print('대각 행렬 S :')
print(S.round(2))

대각 행렬 S :
[[2.69 0.  ]
 [0.   2.05]]


* 상위 2개의 값만 남기고 나머지는 모두 제거된 것을 볼 수 있습니다.
* 이제 직교 행렬 U에 대해서도 2개의 열만 남기고 제거합니다.

In [ ]:
U = U[:,:2] ### ... 이 부분을 완성하시오
print('행렬 U :')
print(U.round(2))

행렬 U :
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


* 2개의 열만 남기고 모두 제거가 된 것을 볼 수 있습니다.
* 이제 행렬 V의 전치 행렬인 VT에 대해서 2개의 행만 남기고 제거합니다.
* 이는 V관점에서는 2개의 열만 남기고 제거한 것이 됩니다.

In [ ]:
VT = VT[:2,:] ### ... 이 부분을 완성하시오
print('직교행렬 VT :')
print(VT.round(2))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


* 이제 축소된 행렬 U, S, VT에 대해서 다시 U × S × VT연산을 하면 기존의 A와는 다른 결과가 나오게 됩니다.
* 값이 손실되었기 때문에 이 세 개의 행렬로는 이제 기존의 A행렬을 복구할 수 없습니다.
* U × S × VT연산을 해서 나오는 값을 A_prime이라 하고 기존의 행렬 A와 값을 비교해보도록 하겠습니다.

In [ ]:
A_prime = np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


* 대체적으로 기존에 0인 값들은 0에 가가운 값이 나오고, 1인 값들은 1에 가까운 값이 나오는 것을 볼 수 있습니다.
* 또한 값이 제대로 복구되지 않은 구간도 존재해보입니다.
<br>

### [차원 축소된 U, S, VT의 크기의 의미]
* 축소된 U는 4 × 2의 크기를 가지는데, 이는 잘 생각해보면 문서의 개수 × 토픽의 수 t의 크기입니다.
    * 단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지되었으니 4개의 문서 각각을 2개의 값으로 표현하고 있습니다.
    * 즉, U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터라고 볼 수 있습니다.
* 축소된 VT는 2 × 9의 크기를 가지는데, 이는 잘 생각해보면 토픽의 수 t × 단어의 개수의 크기입니다.
    * VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터라고 볼 수 있습니다.
<br>

이 문서 벡터들과 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(쿼리)로부터 문서의 유사도를 구하는 것들이 가능해집니다.

### 6.3.1 LSA를 이용한 차원 축소와 성능


In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2000, random_state=7) #압축할 component의 수 지정

X_train_lsa = svd.fit_transform(X_train_tfidf) ### ... 이 부분을 완성하시오
X_test_lsa = svd.transform(X_test_tfidf)

print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))

LSA Converted X shape: (2034, 2000)
Sum of explained variance ratio: 1.000


In [ ]:
LR_clf.fit(X_train_lsa, y_train) ### ... 이 부분을 완성하시오
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_lsa, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_lsa, y_test)))

#Train set score: 0.962
#Test set score: 0.761


In [ ]:
# 100차원으로 압축해보기 -> 성능은 조금 떨어지지만 여전히 높은 성능의 수준이다.
svd = TruncatedSVD(n_components=100, random_state=1) ### ... 이 부분을 완성하시오

X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)

print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))

LR_clf.fit(X_train_lsa, y_train)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_lsa, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_lsa, y_test)))

LSA Converted X shape: (2034, 100)
Sum of explained variance ratio: 0.209
#Train set score: 0.810
#Test set score: 0.745


### 6.3.2 LSA를 이용한 의미 기반의 문서 간 유사도 계산


문서1 : 저는 사과 좋아요
문서2 : 저는 바나나 좋아요
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

띄어쓰기 기준 토큰화를 진행했다고 가정했을 때, 위의 세 문서에 대한 문서-단어 행렬

| 문서 / 단어 | 저는 | 사과 | 좋아요 | 바나나 |
|:--------:|:---:|:---:|:-----:|:----:|
| **문서1**  |  1  |  1  |   1   |   0   |
| **문서2**  |  1  |  0  |   1   |   1   |
| **문서3**  |  2  |  0  |   2   |   2   |

Numpy를 사용해서 코사인 유사도를 계산하는 방법
* cos_sim(A, B) 함수를 구현하고 각 문서 벡터 간의 코사인 유사도를 계산

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B)) ### ... 이 부분을 완성하시오

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


sklearn의 cosine_similarity 를 이용하는 방법

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# DTM 정의
doc1 = np.array([[0,1,1,1]])
doc2 = np.array([[1,0,1,1]])
doc3 = np.array([[2,0,2,2]])

# cosine_similarity를 사용하여 유사도 계산
sim12 = cosine_similarity(doc1, doc2)
sim13 = cosine_similarity(doc1, doc3)
sim23 = cosine_similarity(doc2, doc3)

print('문서 1과 문서2의 유사도 :', sim12[0][0])
print('문서 1과 문서3의 유사도 :', sim13[0][0])
print('문서 2와 문서3의 유사도 :', sim23[0][0])

문서 1과 문서2의 유사도 : 0.6666666666666669
문서 1과 문서3의 유사도 : 0.6666666666666669
문서 2와 문서3의 유사도 : 1.0000000000000002


### LSA 벡터 유사도 측정

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print('#사용된 전체 카테고리:', newsgroups_train.target_names)
print('#첫 문서의 카테고리:', y_train[0])

print('#첫 문서의 내용:', X_train[0])

#변환된 count vector와 기존 값들과의 similarity 계산
sim_result = cosine_similarity([X_train_lsa[0]], X_train_lsa) ### ... 이 부분을 완성하시오

print("#Top 20 유사도(lsa):\n", sorted(sim_result[0].round(2), reverse=True)[:20])
sim_index = (-sim_result[0]).argsort()[:20]
print('#Top 20 유사 뉴스의 인덱스(lsa):\n', sim_index)
sim_labels = [y_train[i] for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(lsa):\n', sim_labels)

#사용된 전체 카테고리: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#첫 문서의 카테고리: 1
#첫 문서의 내용: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Top 20 유사도(lsa):
 [1.0, 0.74, 0.74, 0.72, 0.7, 0.7, 0.69, 0.67, 0.66, 0.65, 0.65, 0.65, 0.63, 0.62, 0.62, 0.62, 0.57, 0.57, 0.55, 0.54]
#Top 20 유사 뉴스의 인덱스(lsa):
 [   0 1957 1674  501 1995 1490  790 1902 1575 1209 1728  892 1892  998
 1038 1826 1290 1089  867  151]
#Top 20 유사 뉴스의 카테

In [ ]:
X_train[1957]

"\n\n\nYou can include postscript epsi files in xfig (encapsulated postscript\ninfo files). You can't actually edit the postscript file, but you're able\nto draw over the postscript file.\n\nThere a eps to epsi converter: eps2epsi (perl program),\n\nSucces,"

In [ ]:
X_train[1674]

'\n\n    The gl2p1.lzh stuff under gfx/show on the Aminet sites includes a\n    utility called pic2hl, that is a filter for HamLab that can handle\n    the most commonly used kinds of .PIC and .CLP files.\n\n    The biggest problem is that the .CLP files don\'t usually contain a\n    palette, so you need to convert a .PIC with the right palette\n    first (which creates a "ram:picpal" file), and then convert the\n    .CLP files.\n'

### TF-IDF 벡터 유사도 측정

In [ ]:
sim_result = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

# 유사도가 높은 순으로 정렬
print("#Top 20 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:20]) ### ... 이 부분을 완성하시오

# 유사도가 높은 순으로 정렬 후, 해당 값의 index를 반환
sim_index = (-sim_result[0]).argsort()[:20] ### ... 이 부분을 완성하시오
print('#Top 20 유사 뉴스의 인덱스(tfidf):\n', sim_index)
sim_labels = [y_train[i] for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(tfidf):\n', sim_labels)

#Top 20 유사도(tfidf):
 [1.0, 0.3, 0.22, 0.21, 0.19, 0.19, 0.19, 0.17, 0.16, 0.16, 0.16, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.14]
#Top 20 유사 뉴스의 인덱스(tfidf):
 [   0 1575 1892 1490  501 1290 1013  998 1636 1705 1995 1957 1664  651
 1038  429 1089 1209 1728 1803]
#Top 20 유사 뉴스의 카테고리(tfidf):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 7-1 주차 수업 내용 -끝-



# [[실습 7-1-1]] 유사도를 이용한 영화 추천 시스템 구현하기: TF-IDF 벡터 vs LSA 벡터
영화 소개, 제목, 평점 등이 있는 **'movies_metadata.csv'** 데이터 셋을 이용하여, 특정 영화와 가장 유사한 영화를 찾아 추천해주기

* 영화 소개(overview)에 대한 내용을 각각 TF-IDF 벡터로도 나타내보고, LSA로도 나타내보기
* 유사한 영화에 자기 자신을 포함할 수 있음. 가능한 경우, 이를 따로 처리해도 좋고 처리하지 않아도 됨
* Dataset: https://www.kaggle.com/rounakbanik/the-movies-dataset
<br>
<br>

### [제출 내용]

1.   TF-IDF를 이용하여, 영화 소개(overview)의 내용이 가장 유사한 영화(title) 5개를 찾는 코드 작성
    
2.   LSA를 이용하여, 영화 소개(overview)의 내용이 가장 유사한 영화(title) 5개를 찾는 코드 작성
3.   TF-IDF와 LSA의 결과 비교 설명. LSA가 의미를 더 잘 찾는 것 같은지? 자신에게는 TF-IDF와 LSA 중 어떤 것이 더 마음에 드는 방법이었는지 결론을 내려보기
    * 유사도, 영화 소개의 내용, average_vote (평점) 등을 참고하여 비교해 보시오.


### [참고 코드]
* **참고용**으로 사용하시면 됩니다. 더 편한 방법이 있다면 해당 방법으로 작성하셔도 좋습니다.
* 참고 코드는 0번째 영화 & TF-IDF에 대해서만 작성되어 있음
    * 0번째 영화가 아닌 다른 n번째 영화를 선택하여 비교해보아도 좋습니다.
    * **(주의!)** LSA에 대한 코드를 추가 작성해야 합니다.

<br>

```
import pandas as pd
df = pd.read_csv('./movies_metadata.csv', encoding='utf-8', low_memory=False)
df.head(5)
```

```
from sklearn.model_selection import train_test_split

# 평점이 8점 이상인 영화 중에서 추천
sampled_df = df[df['vote_average'] >= 8].reset_index(drop=True)
sampled_df.shape
```

```
# overview에 빈 값이 있는 경우, ''로 채우기
sampled_df['overview'] = sampled_df['overview'].fillna('')
```

```
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(sampled_df.overview, sampled_df.title, random_state=0)

tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
```
```
from sklearn.metrics.pairwise import cosine_similarity

print("0번째 영화 제목\n", y_train[0])
print("0번째 영화 소개 내용\n", X_train[0])

sim_result = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

print("#Top 5 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:5])
sim_index = (-sim_result[0]).argsort()[:5]
```
```
result_df = pd.DataFrame({
    '유사도': sorted(sim_result[0].round(2), reverse=True)[:5],
    '인덱스': sim_index,
    '제목': [sampled_df.title[i] for i in sim_index],
    '영화 소개': [sampled_df.overview[i] for i in sim_index],
    '평점': [sampled_df.vote_average[i] for i in sim_index]
})

result_df
```
```
X_train_tfidf.shape
```
```
print('TF-IDF 기반으로 계산한 결과 추천하는 영화는: ', [sampled_df.title[i] for i in sim_index][1:])
```
* ### 위의 참고 코드를 복사 붙여넣기하여 시작해보시면 도움이 될 것 같습니다.
* ### 아래에 코드 또는 텍스트를 추가하여 작성하시기 바랍니다.

In [62]:
import pandas as pd
df = pd.read_csv('./movies_metadata.csv', encoding='utf-8', low_memory=False)
df.head(5)
#df.shape -> (45466, 24)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [63]:
from sklearn.model_selection import train_test_split

# 평점이 8점 이상인 영화 중에서 추천
sampled_df = df[df['vote_average'] >= 8].reset_index(drop=True)
print(sampled_df.shape)

# overview에 빈 값이 있는 경우, ''로 채우기
sampled_df['overview'] = sampled_df['overview'].fillna('')

(1895, 24)


In [78]:
############ !!! tfdif로 유사한 영화 5개 출력하기 !!! #############
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(sampled_df.overview, sampled_df.title, random_state=0)

tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

print("0번째 영화 제목\n", y_train[0])
print("0번째 영화 소개 내용\n", X_train[0])

sim_result = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

print("#Top 5 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:5])
sim_index = (-sim_result[0]).argsort()[:5]

result_df = pd.DataFrame({
    '유사도': sorted(sim_result[0].round(2), reverse=True)[:5],
    '인덱스': sim_index,
    '제목': [sampled_df.title[i] for i in sim_index],
    '영화 소개': [sampled_df.overview[i] for i in sim_index],
    '평점': [sampled_df.vote_average[i] for i in sim_index]
})

result_df
X_train_tfidf.shape

print('TF-IDF 기반으로 계산한 결과 추천하는 영화는: ', [sampled_df.title[i] for i in sim_index][1:])
print(sim_index[1:])

0번째 영화 제목
 Se7en
0번째 영화 소개 내용
 Two homicide detectives are on a desperate hunt for a serial killer whose crimes are based on the "seven deadly sins" in this dark and haunting film that takes viewers from the tortured remains of one victim to the next. The seasoned Det. Sommerset researches each sin in an effort to get inside the killer's mind, while his novice partner, Mills, scoffs at his efforts to unravel the case.
#Top 5 유사도(tfidf):
 [1.0, 0.27, 0.26, 0.23, 0.23]
TF-IDF 기반으로 계산한 결과 추천하는 영화는:  ['Pin Boy', 'Karachi se Lahore', 'Houston, We Have a Problem!', 'Pudhupettai']
[1301 1130 1399 1003]


In [84]:
sampled_df.overview[1130]

'A road trip from Karachi to Lahore where 5 friends discover themselves and the country amidst getting to their destination.'

 Two homicide detectives are on a desperate hunt for a serial killer whose crimes are based on the "seven deadly sins" in this dark and haunting film that takes viewers from the tortured remains of one victim to the next. The seasoned Det. Sommerset researches each sin in an effort to get inside the killer's mind, while his novice partner, Mills, scoffs at his efforts to unravel the case.


In [81]:
############ !!! LSA로 유사한 영화 5개 출력하기 !!! #############

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, random_state=7) #압축할 component의 수 지정

X_train_lsa = svd.fit_transform(X_train_tfidf) ### ... 이 부분을 완성하시오
X_test_lsa = svd.transform(X_test_tfidf)

print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))

LSA Converted X shape: (1421, 100)
Sum of explained variance ratio: 0.301


In [82]:
from sklearn.metrics.pairwise import cosine_similarity

print("0번째 영화 제목\n", y_train[0])
print("0번째 영화 소개 내용\n", X_train[0])

#변환된 count vector와 기존 값들과의 similarity 계산
sim_result = cosine_similarity([X_train_lsa[0]], X_train_lsa)

print("#Top 5 유사도(lsa):\n", sorted(sim_result[0].round(2), reverse=True)[:5])
sim_index = (-sim_result[0]).argsort()[:5]

result_df = pd.DataFrame({
    '유사도': sorted(sim_result[0].round(2), reverse=True)[:5],
    '인덱스': sim_index,
    '제목': [sampled_df.title[i] for i in sim_index],
    '영화 소개': [sampled_df.overview[i] for i in sim_index],
    '평점': [sampled_df.vote_average[i] for i in sim_index]
})

result_df
X_train_tfidf.shape

print('TF-IDF 기반으로 계산한 결과 추천하는 영화는: ', [sampled_df.title[i] for i in sim_index][1:])
print(sim_index[1:])

0번째 영화 제목
 Se7en
0번째 영화 소개 내용
 Two homicide detectives are on a desperate hunt for a serial killer whose crimes are based on the "seven deadly sins" in this dark and haunting film that takes viewers from the tortured remains of one victim to the next. The seasoned Det. Sommerset researches each sin in an effort to get inside the killer's mind, while his novice partner, Mills, scoffs at his efforts to unravel the case.
#Top 5 유사도(lsa):
 [1.0, 0.65, 0.61, 0.6, 0.55]
TF-IDF 기반으로 계산한 결과 추천하는 영화는:  ['Pin Boy', 'The Dream Team', 'Houston, We Have a Problem!', 'One Last Hug']
[1301 1044 1399  914]


In [85]:
sampled_df.overview[1044]

'The world had rarely seen a frenzy as the one the Dream Team created when it arrived in Barcelona, Spain, in July 1992. The Dream Team featured 11 future Naismith Memorial Basketball Hall of Fame players and three future Hall of Famers on the coaching staff, including head coach Chuck Daly.'

* TF-IDF와 LSA의 결과 비교 설명. LSA가 의미를 더 잘 찾는 것 같은지? 자신에게는 TF-IDF와 LSA 중 어떤 것이 더 마음에 드는 방법이었는지 결론을 내려보기

* 유사도, 영화 소개의 내용, average_vote (평점) 등을 참고하여 비교해 보시오.

### 0번째 영화로 tfidf와 LSA 결과 비교 (n_components=100으로 설정)
* 0번째 영화 소개 내용
 Two homicide detectives are on a desperate hunt for a serial killer whose crimes are based on the "seven deadly sins" in this dark and haunting film that takes viewers from the tortured remains of one victim to the next. The seasoned Det. Sommerset researches each sin in an effort to get inside the killer's mind, while his novice partner, Mills, scoffs at his efforts to unravel the case.

* 유사도 비교
> tfidf Top 5 유사도(tfidf):
 [1.0, 0.27, 0.26, 0.23, 0.23]

  > LSA Top 5 유사도(lsa):
 [1.0, 0.65, 0.61, 0.6, 0.55]

* 유사도 높은 영화 overview 비교
> (tfidf) sampled_df.overview[1130] A road trip from Karachi to Lahore where 5 friends discover themselves and the country amidst getting to their destination.

  > (LSA) sampled_df.overview[1044] The world had rarely seen a frenzy as the one the Dream Team created when it arrived in Barcelona, Spain, in July 1992. The Dream Team featured 11 future Naismith Memorial Basketball Hall of Fame players and three future Hall of Famers on the coaching staff, including head coach Chuck Daly.

* tfidf 와 LSA의 유사도로 보면 LSA의 유사도가 훨씬 높은 것을 알 수 있다. 영화 내용 소개로 보았을 때, LSA가 차원 수를 100으로 줄였음에도 의미를 더 잘 찾은 것 같고, 더 빨라서 좋았다.

---

# [파일] -> [다운로드] -> [.ipynb 다운로드]
# 제출 후 **구글 계정 로그아웃** 잘 하시기 바랍니다.